In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import mlflow
import pkg_resources

from mlflow.types.responses import ResponsesAgentRequest

from agent import LLM_ENDPOINT_NAME, AGENT

In [0]:
input_example = {"input": [{"role": "user", "content": "How many Rs in strawberry?"}]}
# input_example = ResponsesAgentRequest(input=[{"role": "user", "content": "How many Rs in strawberry?"}])

In [0]:
with mlflow.start_run():
    AGENT.predict(input_example)

In [0]:
resources = [mlflow.models.resources.DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]

In [0]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Workspace/Users/stuart.lynn@databricks.com/u2m-experiment")

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            f"databricks-langchain=={pkg_resources.get_distribution('databricks-langchain').version}",
            f"langchain-core=={pkg_resources.get_distribution('langchain-core').version}",
            f"databricks-sdk=={pkg_resources.get_distribution('databricks-sdk').version}",
            f"mlflow[databricks]=={pkg_resources.get_distribution('mlflow').version}",
        ]
    )

In [0]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{logged_agent_info.run_id}/agent")
loaded_model.predict(input_example)

In [0]:
mlflow.models.predict(logged_agent_info.model_uri, input_example, env_manager="uv")